In [129]:
import pandas as pd 
import numpy as np 

In [130]:
car = pd.read_csv('car.csv')

In [131]:
car.shape

(892, 6)

## Quality 
-- Year has many non-year values
-- year object to int
-- price has ask for price 
-- price oject to int 
-- kmh driven object to int and nan value 
-- fuel_type has nan value 


## Cleaning


In [132]:
backup = car.copy()

In [133]:
car = car[car['year'].str.isnumeric()]

In [134]:
car['year'] = car['year'].astype(int)

In [135]:
car = car[car['Price'] != "Ask For Price"]

In [136]:
car['Price'] = car['Price'] .str.replace(',',"").astype(int)

In [137]:
car['kms_driven'] = car['kms_driven'].str.split(' ').str.get(0).str.replace(',','')


In [138]:
car['kms_driven']

0       45000
1          40
3       28000
4       36000
6       41000
        ...  
886    132000
888     27000
889     40000
890    Petrol
891    Petrol
Name: kms_driven, Length: 819, dtype: object

In [139]:
car=car[car['kms_driven'].str.isnumeric()]
car['kms_driven'] = car['kms_driven'].astype(int)

In [140]:
car.info()

<class 'pandas.core.frame.DataFrame'>
Index: 817 entries, 0 to 889
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   name        817 non-null    object
 1   company     817 non-null    object
 2   year        817 non-null    int32 
 3   Price       817 non-null    int32 
 4   kms_driven  817 non-null    int32 
 5   fuel_type   816 non-null    object
dtypes: int32(3), object(3)
memory usage: 35.1+ KB


In [141]:
car['name']=car['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [142]:
car.head()

,name,company,year,Price,kms_driven,fuel_type
0,Hyundai Santro Xing,Hyundai,2007,80000,45000,Petrol
1,Mahindra Jeep CL550,Mahindra,2006,425000,40,Diesel
3,Hyundai Grand i10,Hyundai,2014,325000,28000,Petrol
4,Ford EcoSport Titanium,Ford,2014,575000,36000,Diesel
6,Ford Figo,Ford,2012,175000,41000,Diesel


In [143]:
car = car.reset_index(drop=True)

In [144]:
car.describe()

,year,Price,kms_driven
count,817.000000,8.170000e+02,817.000000
mean,2012.440636,4.115503e+05,46250.714810
std,4.002354,4.749173e+05,34283.745254
min,1995.000000,3.000000e+04,0.000000
25%,2010.000000,1.750000e+05,27000.000000
50%,2013.000000,2.999990e+05,41000.000000
75%,2015.000000,4.900000e+05,56758.000000
max,2019.000000,8.500003e+06,400000.000000


In [145]:
car[car['Price']>6e6]

,name,company,year,Price,kms_driven,fuel_type
535,Mahindra XUV500 W6,Mahindra,2014,8500003,45000,Diesel


In [146]:
car.to_csv('cleaned_car.csv')

## Model

In [147]:
x = car.drop(columns='Price')
y = car['Price']

In [148]:
from sklearn.model_selection import train_test_split 
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)

In [149]:
from sklearn.linear_model import LinearRegression 
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer 
from sklearn.pipeline import make_pipeline

In [150]:
ohe = OneHotEncoder()
ohe.fit(x[['name','company','fuel_type']])

OneHotEncoder()

In [151]:
column_trans = make_column_transformer((OneHotEncoder(categories=ohe.categories_),['name','company','fuel_type']), remainder='passthrough')

In [152]:
lr = LinearRegression()

In [153]:
pipe = make_pipeline(column_trans,lr)

In [154]:
pipe.fit(x_train,y_train)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\compose\_column_transformer.py:1623: FutureWarning: 
The format of the columns of the 'remainder' transformer in ColumnTransformer.transformers_ will change in version 1.7 to match the format of the other transformers.
At the moment the remainder columns are stored as indices (of type int). With the same ColumnTransformer configuration, in the future they will be stored as column names (of type str).
To use the new behavior now and suppress this warning, use ColumnTransformer(force_int_remainder_cols=False).

  warnings.warn(


Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(categories=[array(['Audi A3 Cabriolet', 'Audi A4 1.8', 'Audi A4 2.0', 'Audi A6 2.0',
       'Audi A8', 'Audi Q3 2.0', 'Audi Q5 2.0', 'Audi Q7', 'BMW 3 Series',
       'BMW 5 Series', 'BMW 7 Series', 'BMW X1', 'BMW X1 sDrive20d',
       'BMW X1 xDrive20d', 'Chevrolet Beat', 'Chevrolet Beat...
                                                                            array(['Audi', 'BMW', 'Chevrolet', 'Datsun', 'Fiat', 'Force', 'Ford',
       'Hindustan', 'Honda', 'Hyundai', 'Jaguar', 'Jeep', 'Land',
       'Mahindra', 'Maruti', 'Mercedes', 'Mini', 'Mitsubishi', 'Nissan',
       'Renault', 'Skoda', 'Tata', 'Toyota', 'Volkswagen', 'Volvo'],
      dtype=object),
                                                                            array(['Diesel', 'LPG', 'Petrol', nan], dtype=object)]),
                                                  ['name', 'company',
                                                   'fuel_type'])])),
                ('linearregression', LinearRegression())])

In [155]:
y_pred = pipe.predict(x_test)

In [156]:
r2_score(y_test,y_pred)

0.8037868874501772

In [162]:
scores=[]
for i in range(1000):
    x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=i)
    lr = LinearRegression()
    pipe = make_pipeline(column_trans,lr)
    pipe.fit(x_train,y_train)
    y_pred = pipe.predict(x_test)
    scores.append(r2_score(y_test,y_pred))

In [163]:
np.argmax(scores)

653

In [165]:
scores[np.argmax(scores)]

0.8176307384150114

In [167]:
x_train , x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=np.argmax(scores))
lr = LinearRegression()
pipe = make_pipeline(column_trans,lr)
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
r2_score(y_test,y_pred)

0.8176307384150114

In [173]:
import pickle
pickle.dump(pipe,open('car_model.pkl','wb'))

In [179]:
pipe.predict(pd.DataFrame([['Maruti Suzuki Swift','Maruti',2019,100,'Petrol']], columns =['name','company','year','kms_driven','fuel_type']))

array([436818.42201649])